<a href="https://colab.research.google.com/github/sujinee01/Capstone/blob/main/0%EB%8F%84_all_cycle_tcn_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0. 라이브러리**

In [83]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import matplotlib.pyplot as plt
import seaborn as sns
import os
from keras.preprocessing.sequence import pad_sequences

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [84]:
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import MinMaxScaler
# Min-Max 스케일러 인스턴스 생성
scaler = MinMaxScaler()

# **1. 데이터 정제**

결측치 처리, 이상치 제거, 데이터 타입 변환 등 데이터를 분석에 적합한 형태로 정제합니다.

In [85]:
# 데이터 정제 함수
def preprocess_excel(file_path):
    # 엑셀 파일 로드
    df = pd.read_excel(file_path)

    # 불필요한 열 제거
    df.drop(columns=['Step_Time(s)','Step_Index','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 10'], inplace=True)

    # 결측치가 있는 행 제거
    df.dropna(inplace=True)

    return df

In [86]:
# Driving_Cycle 특성 추가 함수
import re

#파일 이름에서 정보 추출하는 함수 정의
def extract_info_from_filename(filename):
    # Driving Cycle과 온도 정보를 추출하는 정규 표현식
    pattern = r'LiFePO4_(\w+)_SOC_(-?\d+)_'
    match = re.search(pattern, filename)
    if match:
        Driving_Cycle = match.group(1)
        temperature = int(match.group(2))
        return Driving_Cycle, temperature
    else:
        return None, None

# Driving_Cycle을 정수형으로 반환하는 함수
def Driving_Cycle_Numb(Driving_Cycle):
  if Driving_Cycle=='DST':
    Numb=1
    return Numb
  elif Driving_Cycle=='FUDS':
    Numb=2
    return Numb
  elif Driving_Cycle=='US06':
    Numb=3
    return Numb

In [87]:
def process_all_files(folder_path):
    # 모든 엑셀 파일 목록 가져오기
    file_names = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]

    # 파일 목록을 원하는 순서로 정렬

    # 모든 파일에 대해 전처리 수행
    processed_dfs = []
    for file_name in file_names:
        file_path = os.path.join(folder_path, file_name)

        # 불필요한 열, 결측치 행 제거
        processed_df = preprocess_excel(file_path)

        # 차분 : 비정상 시계열 -> 정상시계열
        result = adfuller(processed_df['SOC(t)'])
        processed_df['SOC(t)_diff'] = processed_df['SOC(t)'].diff()
        processed_df['SOC(t)_diff'][0]=processed_df['SOC(t)_diff'][1] # NaN 값 대체

        # 주행 사이클, 온도 열 추가
        Driving_Cycle, temperature = extract_info_from_filename(file_name)
        processed_df['Driving_Cycle']= Driving_Cycle_Numb(Driving_Cycle)

        processed_dfs.append(processed_df)

    return processed_dfs

In [88]:
folder_path = '/content/drive/MyDrive/LiFePO4 Dynamic Profile Files/LiFePO4 Dynamic Profile Files'
all_processed_data = process_all_files(folder_path)

In [89]:
all_processed_data

[]

In [90]:
type(all_processed_data)

list

# **3. 특성 엔지니어링 및 전처리**

# **2. 탐색적 데이터 분석(EDA)**

**-시각화** : 시계열 데이터의 추세(trend), 계절성(seasonality), 순환성(cyclic) 등을 파악하기 위해 시각화를 수행합니다.

In [91]:
# 시각화를 위한 데이터 준비
i=1
for df in all_processed_data:
  time_column = df['Test_Time(s)']
  current = df['Current(A)']
  voltage = df['Voltage(V)']
  temperature = df['Temperature (C)_1']
  soc_diff = df['SOC(t)_diff']
  soc = df['SOC(t)']

  # 각 값에 대한 시간 변화를 선 그래프로 시각화
  plt.figure(figsize=(20, 10))

  plt.subplot(6, 5, i)
  plt.plot(time_column, current, label='Current(A)')
  plt.xlabel('Time')
  plt.ylabel('Current (A)')
  plt.title('Current over Time')
  plt.legend()

  plt.subplot(6, 5, i+1)
  plt.plot(time_column, voltage, label='Voltage(V)', color='orange')
  plt.xlabel('Time')
  plt.ylabel('Voltage (V)')
  plt.title('Voltage over Time')
  plt.legend()

  plt.subplot(6, 5, i+2)
  plt.plot(time_column, temperature, label='Temperature (C)', color='green')
  plt.xlabel('Time')
  plt.ylabel('Temperature (C)')
  plt.title('Temperature over Time')
  plt.legend()

  plt.subplot(6, 5, i+3)
  plt.plot(time_column, soc_diff, label='SOC(t)_diff', color='red')
  plt.xlabel('Time')
  plt.ylabel('SOC_diff (%)')
  plt.title('SOC_diff over Time')
  plt.legend()

  plt.subplot(6, 5, i+4)
  plt.plot(time_column, soc, label='SOC(t)', color='red')
  plt.xlabel('Time')
  plt.ylabel('SOC(t) (%)')
  plt.title('SOC(t) over Time')
  plt.legend()

  plt.tight_layout()
  plt.show()

  i=i+5

**-통계 분석** : 데이터의 통계적 속성을 분석합니다(예: 평균, 중앙값, 표준편차 등).

**-데이터 분할** : 모델 학습을 위해 데이터를 훈련 세트와 테스트 세트로 분할합니다.

In [92]:
# 데이터 분할

# 특성과 타겟을 분리하는 함수
def split_features_targets(df):
    features = df[['Test_Time(s)', 'Current(A)', 'Voltage(V)', 'Temperature (C)_1', 'Driving_Cycle']]
    targets = df['SOC(t)']
    return features, targets

# 전체 데이터셋에서 특성과 타겟 분리
features_targets = [split_features_targets(df) for df in all_processed_data]

In [93]:
print(features_targets)

[]


In [94]:
# train/validation/test split

train_set = features_targets[1:6:2]
validation_set = features_targets[2:5:2]
test_set = features_targets[5:]

# 각 세트의 크기 확인
print(f"Train Set Size: {len(train_set)}")
print(f"Validation Set Size: {len(validation_set)}")
print(f"Test Set Size: {len(test_set)}")

Train Set Size: 0
Validation Set Size: 0
Test Set Size: 0


In [95]:
# 결과 출력
print('Train_set')
print(train_set)
print('Validation_set')
print(validation_set)
print('Test_set')
print(test_set)

Train_set
[]
Validation_set
[]
Test_set
[]


In [96]:
# MinMax Scailing

# 특성과 타겟을 위한 별도의 스케일러 생성
feature_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()

# Train 데이터에 스케일링 fit 및 transform 적용
scaled_train_set = []
for features, targets in train_set:
    scaled_features = pd.DataFrame(feature_scaler.fit_transform(features), columns=features.columns, index=features.index)
    scaled_targets = pd.Series(target_scaler.fit_transform(targets.values.reshape(-1, 1)).flatten(), index=targets.index)
    scaled_train_set.append((scaled_features, scaled_targets))

# Validation 및 Test 데이터에 스케일링 transform 적용
def apply_transform(data_set, feature_scaler, target_scaler):
    transformed_data_set = []
    for features, targets in data_set:
        transformed_features = pd.DataFrame(feature_scaler.transform(features), columns=features.columns, index=features.index)
        transformed_targets = pd.Series(target_scaler.transform(targets.values.reshape(-1, 1)).flatten(), index=targets.index)
        transformed_data_set.append((transformed_features, transformed_targets))
    return transformed_data_set

scaled_validation_set = apply_transform(validation_set, feature_scaler, target_scaler)
scaled_test_set = apply_transform(test_set, feature_scaler, target_scaler)

In [97]:
print('Scaled_train_set\n', scaled_train_set)
print('Scaled_validation_set\n', scaled_validation_set)
print('Scaled_test_set\n', scaled_test_set)

Scaled_train_set
 []
Scaled_validation_set
 []
Scaled_test_set
 []


Driving_Cycle은 범주형 변수이므로 원-핫 인코딩을 적용하는 것이 적절하다. 바꿔야함!

# **4. 모델 선택 및 학습**

**-모델 선택**: ARIMA, SARIMA, Prophet, LSTM, GRU, TCN 등과 같은 다양한 시계열 예측 모델 중 프로젝트 목표와 데이터 특성에 맞는 모델을 선택합니다.

In [98]:
# !pip install git+https://github.com/philipperemy/keras-tcn.git


In [99]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tcn import TCN, tcn_full_summary
from tensorflow.keras.optimizers import Adam
import numpy as np

**패딩의 주의점**


패딩을 사용할 때 padding='post'는 시퀀스의 끝에 0을 추가하며, dtype='float32'는 모델이 요구하는 데이터 타입에 맞춰 설정해야 합니다. 또한, 패딩이 모델의 예측 성능에 영향을 줄 수 있으므로, TCN 같은 네트워크에서는 패딩이 성능에 미치는 영향을 주의 깊게 관찰할 필요가 있습니다.

In [100]:
# 모든 데이터셋에서 가장 긴 시퀀스 길이를 찾습니다.
max_len = max([max(len(features), len(targets)) for features, targets in scaled_train_set + scaled_validation_set + scaled_test_set])

# 데이터를 패딩하여 동일한 길이로 만듭니다.
X_train_padded = np.array([np.pad(features.values, ((0, max_len - len(features)), (0, 0)), mode='constant') for features, _ in scaled_train_set])
y_train_padded = np.array([np.pad(targets.values, (0, max_len - len(targets)), mode='constant') for _, targets in scaled_train_set])

X_val_padded = np.array([np.pad(features.values, ((0, max_len - len(features)), (0, 0)), mode='constant') for features, _ in scaled_validation_set])
y_val_padded = np.array([np.pad(targets.values, (0, max_len - len(targets)), mode='constant') for _, targets in scaled_validation_set])

X_test_padded = np.array([np.pad(features.values, ((0, max_len - len(features)), (0, 0)), mode='constant') for features, _ in scaled_test_set])
y_test_padded = np.array([np.pad(targets.values, (0, max_len - len(targets)), mode='constant') for _, targets in scaled_test_set])

# 데이터의 형태 확인
print("X_train_padded shape:", X_train_padded.shape)
print("y_train_padded shape:", y_train_padded.shape)
print("X_val_padded shape:", X_val_padded.shape)
print("y_val_padded shape:", y_val_padded.shape)
print("X_test_padded shape:", X_test_padded.shape)
print("y_test_padded shape:", y_test_padded.shape)

ValueError: max() arg is an empty sequence

In [ ]:
print(type(X_train_padded))

In [ ]:
# 결과 출력
print("Padded X_train:")
print(X_train_padded)
print("Padded y_train:")
print(y_train_padded)
print("Padded X_val:")
print(X_val_padded)
print("Padded y_val:")
print(y_val_padded)
print("Padded X_test:")
print(X_test_padded)
print("Padded y_test:")
print(y_test_padded)

**-모델 학습 및 최적화** : 선택한 모델을 훈련 데이터에 적용하여 학습시킵니다.

In [ ]:
pip install keras-tcn  # TCN 라이브러리 설치

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tcn import TCN, tcn_full_summary
from tensorflow.keras.optimizers import Adam

In [ ]:
# 모델 구성
model = Sequential([
    TCN(input_shape=(None, 5),  # 입력 시퀀스 길이는 가변적이며, 특성 수는 5
        nb_filters=64,
        kernel_size=3,  # 커널 사이즈는 3으로 설정
        nb_stacks=1,
        dilations=[1, 2, 4, 8, 16],
        padding='causal',
        use_skip_connections=True,
        dropout_rate=0.2,  # 드롭아웃 추가
        return_sequences=True,  # 시퀀스의 모든 타임 스텝에 대한 출력을 반환
        activation='relu',
        kernel_initializer='he_normal'),
    Dense(1)  # 출력층은 하나의 값을 예측
])

In [ ]:
# 모델 요약 출력
model.summary()

In [ ]:
# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='mean_squared_error',
              metrics=['mae'])  # 평균 절대 오차도 추적

In [ ]:
# 모델 학습
history = model.fit(X_train_padded, y_train_padded,
                    epochs=10,
                    batch_size=32,
                    validation_data=(X_val_padded, y_val_padded))

In [ ]:
# 학습 과정 시각화
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# 정규화된 훈련 데이터를 원래 스케일로 되돌리기
def inverse_transform(y_pred, scaler):
    y_pred_rescaled = scaler.inverse_transform(y_pred)
    return y_pred_rescaled

# 오차율 계산 함수
def calculate_error_rate(y_true, y_pred):
    # 0으로 나누기 오류를 방지하기 위해 0인 경우에는 0을 반환하고, 그 외에는 오차율을 계산합니다.
    mask = y_true != 0
    if mask.any():
        return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    else:
        return np.nan  # 0으로 나누기 오류가 발생한 경우에는 NaN을 반환합니다.



In [ ]:
# 모델을 사용하여 예측 수행
y_pred_padded = model.predict(X_test_padded)

In [ ]:
y_pred_padded

In [ ]:
y_test_padded

In [ ]:
# 계산된 오차율
error_rate = calculate_error_rate(y_test_padded, y_pred_padded)

# 결과값 출력
print(f'Error Rate: {error_rate:.2f}%')



In [ ]:
# 역변환

In [ ]:
# 오차율 계산
error_rate = calculate_error_rate(y_true_rescaled, y_pred_rescaled)
print(f'Error Rate: {error_rate:.2f}%')

# **5. 모델 평가 및 선택**

**-성능 평가** : MAE, RMSE, MAPE 등의 지표를 사용하여 모델의 예측 성능을 평가합니다.

# **6. 예측 및 결과 해석**

**-미래 예측**: 최종 모델을 사용하여 미래 값에 대한 예측을 수행합니다.

**-결과 해석**: 예측 결과를 분석하고, 비즈니스 인사이트나 결정을 내리는 데 활용합니다.